In [105]:
import pandas as pd
import xgboost as xgb


df_test = pd.read_csv('released_dataset/test-classification-round3.dat', header=None)
df_test.columns = ['username']
df_test

,username
0,livapastanesi
1,barisgross
2,tusasshop
3,etolyadigital
4,tugrulonur
...,...
995,utkucubukcuoglu
996,ucan_baba
997,rehberle
998,ntaslicay


In [106]:
df_train = pd.read_parquet("data/training-dataset.parquet")
df_train

,username,biography,bio_links,category_enum,label,posts
0,deparmedya,#mediaplanning #mediabuying #sosyalmedya,[http://www.deparmedya.com/],LOCAL,tech,"[Cumhuriyetimizin 100.yılı kutlu olsun♾️🇹🇷, Or..."
1,beyazyakaliyiz,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,[http://yinemipazartesi.wordpress.com/],PERSONAL_BLOG,None,[Bu diyaloğun yaşanmadığı bir online toplantı ...
2,kafesfirin,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,[http://www.kafesfirin.com/],BRAND,food,[Bugün bir fincan köpüklü Türk kahvesiyle taçl...
3,vimerang,Dijital İletişim Yönetimi🎬info@vimerang.comq,[http://vimerang.com/],VIDEO_CREATOR,tech,[Saygı ve özlemle🖤\n\n#atatürk #10kasım #10kas...
4,totalenergies_istasyonlari,TotalEnergies İstasyonları resmi Instagram hes...,[https://bit.ly/45XDUDr],ENERGY_COMPANY,None,[Başöğretmenimiz Gazi Mustafa Kemal Atatürk’ün...
...,...,...,...,...,...,...
5410,herbisiatolye,🌱 Şifalı Bitkiler Atölyesi 🌾\n📲 WhatsApp Sipar...,[http://www.herbisi.com.tr/],HEALTH_BEAUTY,health and lifestyle,[Gurdjieff kutsal dansları ve öğretisi çalışma...
5411,nestleturkiye,"117 YILDIR YUVAMIZ TÜRKİYE! \n5 fabrikamız, 30...",[https://www.nestle.com.tr/hakkimizda],FOOD_COMPANY,None,[Aramızdan ayrılışının 85. yılında Gazi Mustaf...
5412,tariszeytinyagi,None,[http://www.tariszeytin.com.tr],ACTIVITY_GENERAL,None,"[DÜNYA DİYABET GÜNÜ\n\nŞeker hastalığı, vücudu..."
5413,iklim_ce,👇👇İklimce Sohbetler’le ilgili her şey,[https://linktr.ee/iklim_ce],PERSON,None,[Enerji ve Tabii Kaynaklar Bakanlığı’nın açıkl...


In [107]:
from sklearn.preprocessing import TargetEncoder

df_enum_lab = df_train[['label','category_enum']]
df_enum_lab = df_enum_lab.dropna(subset=["label"])

df_enum_lab["category_enum"] = df_enum_lab["category_enum"].fillna("Missing")

target_encoder = TargetEncoder()

target_encoder.fit(df_enum_lab["category_enum"].to_numpy().reshape(-1, 1), df_enum_lab["label"].tolist())

TargetEncoder()

In [108]:
from consts import *

df_embeds = pd.read_parquet(get_filename(TT.ALL, EMBED_PROVIDER.OPENAI, MEAN_METHOD.CENTROID))
df_embeds

,username,embeds_wavg
0,01burdaavm,"[-0.007527757387694456, -0.010574967835974497,..."
1,1001sanat,"[-0.011305028517603753, -0.010828658876669313,..."
2,1924istanbul,"[-0.014803369030533694, 4.9135569503722457e-05..."
3,1983beyoglu,"[-0.005243633780651946, -0.010119351362150327,..."
4,1dil1insan,"[-0.016736346542457883, 0.02081970183089783, -..."
...,...,...
3469,zpor,"[0.007829824360708068, 0.023413542714805335, -..."
3470,zuhaldentakilar,"[-0.011644816969826801, -0.013427980506543428,..."
3471,zulaoyun,"[0.007932599635500977, -0.016169525521230025, ..."
3472,zulaoyunespor,"[0.0034837810362845084, -0.011363345894454155,..."


In [109]:
df_test_m1 = pd.merge(df_test,df_train, how='left', on='username')
df_test_m1

,username,biography,bio_links,category_enum,label,posts
0,livapastanesi,Çukurambar/Farabi/A.City/Yıldız/Ayrancı/Taurus...,[https://www.livashop.com/],DESSERT_RESTAURANT,None,"["" Atatürk, bir lider olmanın çok daha ötesind..."
1,barisgross,%100 Yerli Besi Güvenilir Et\nTaptaze Meyve v...,[https://barisgrossonlinemagaza.com/],RETAIL_COMPANY,food,[Halk Günü Fırsatlarını Kaçırma !\n🍅🥬🌶️🍋🫑🌽🥑🍑\n...
2,tusasshop,Türk Havacılık Uzay Sanayii Lisanslı Ürünler...,[http://www.tusasshop.com.tr/],PRODUCT_SERVICE,None,"[Hiç durmadan, hiç yorulmadan, daima gösterdiğ..."
3,etolyadigital,#GücünüKeşfet\nMarkaları geliştiren dijital pa...,[https://etolyadigital.com/tr],MARKETING_AGENCY,None,[Devamlılık başarıyı getirir. 💪\n\n#webajans #...
4,tugrulonur,Genel Yayın Yönetmeni ve Moderatör @voleapp \n...,"[, http://www.twitter.com/onurtugrul]",JOURNALIST,entertainment,"[Hull City’ye uğurlu geldim. Takım, üstün oyna..."
...,...,...,...,...,...,...
995,utkucubukcuoglu,"Oyuncu, Eğitmen, Oyuncu koçu, Seslendirmen\n@u...",[],ACTOR,health and lifestyle,[Evliliğimizin tam 4.yılına gelen en güzel hed...
996,ucan_baba,"Yedek ebeveyn değil, babayım!",[https://www.youtube.com/channel/UCZKim0gdIoM6...,DIGITAL_CREATOR,None,"[3 yıl oldu. Çoğiyi değil mi? ozumus, İyi ki d..."
997,rehberle,"Doğa, Mimarlık ve Arkeoloji Sevdalısı Bir Tur ...","[http://bit.ly/41CQOFM, http://bit.ly/3RfF09d]",TOUR_GUIDE,None,[Bu gölü tanıyabildiniz mi❓\n\n📌 Türkiye’nin e...
998,ntaslicay,MHP Merkez Yönetim Kurulu Üyesi \n27. Dönem MH...,[],POLITICIAN,health and lifestyle,"[Bu davanın parmak izidir onlar,\nBu davanın c..."


In [110]:
df_test_m2 = pd.merge(df_test_m1, df_embeds, how='left', on='username')
df_test_m2

,username,biography,bio_links,category_enum,label,posts,embeds_wavg
0,livapastanesi,Çukurambar/Farabi/A.City/Yıldız/Ayrancı/Taurus...,[https://www.livashop.com/],DESSERT_RESTAURANT,None,"["" Atatürk, bir lider olmanın çok daha ötesind...","[-0.021919882393740906, 0.011356396167832213, ..."
1,barisgross,%100 Yerli Besi Güvenilir Et\nTaptaze Meyve v...,[https://barisgrossonlinemagaza.com/],RETAIL_COMPANY,food,[Halk Günü Fırsatlarını Kaçırma !\n🍅🥬🌶️🍋🫑🌽🥑🍑\n...,"[-0.003867788702089352, 0.004385191196449242, ..."
2,tusasshop,Türk Havacılık Uzay Sanayii Lisanslı Ürünler...,[http://www.tusasshop.com.tr/],PRODUCT_SERVICE,None,"[Hiç durmadan, hiç yorulmadan, daima gösterdiğ...","[-0.019723006614887892, 0.004805522574673402, ..."
3,etolyadigital,#GücünüKeşfet\nMarkaları geliştiren dijital pa...,[https://etolyadigital.com/tr],MARKETING_AGENCY,None,[Devamlılık başarıyı getirir. 💪\n\n#webajans #...,"[-0.01758344073235769, -0.011251978524460625, ..."
4,tugrulonur,Genel Yayın Yönetmeni ve Moderatör @voleapp \n...,"[, http://www.twitter.com/onurtugrul]",JOURNALIST,entertainment,"[Hull City’ye uğurlu geldim. Takım, üstün oyna...","[0.002862689993879321, 0.004521826552886835, -..."
...,...,...,...,...,...,...,...
995,utkucubukcuoglu,"Oyuncu, Eğitmen, Oyuncu koçu, Seslendirmen\n@u...",[],ACTOR,health and lifestyle,[Evliliğimizin tam 4.yılına gelen en güzel hed...,"[-0.0007412925928566927, 0.0038359375974205715..."
996,ucan_baba,"Yedek ebeveyn değil, babayım!",[https://www.youtube.com/channel/UCZKim0gdIoM6...,DIGITAL_CREATOR,None,"[3 yıl oldu. Çoğiyi değil mi? ozumus, İyi ki d...","[0.005686445175323538, -0.006737246289064363, ..."
997,rehberle,"Doğa, Mimarlık ve Arkeoloji Sevdalısı Bir Tur ...","[http://bit.ly/41CQOFM, http://bit.ly/3RfF09d]",TOUR_GUIDE,None,[Bu gölü tanıyabildiniz mi❓\n\n📌 Türkiye’nin e...,"[-0.00048104971479939805, 0.002966425380183890..."
998,ntaslicay,MHP Merkez Yönetim Kurulu Üyesi \n27. Dönem MH...,[],POLITICIAN,health and lifestyle,"[Bu davanın parmak izidir onlar,\nBu davanın c...","[-0.0008403996685689565, -0.011229726112891414..."


In [111]:
df_test_correct = df_test_m2[~df_test_m2['label'].isna()]
df_test_correct = df_test_correct[['username','label']].reset_index(drop=True)
df_test_correct

,username,label
0,barisgross,food
1,tugrulonur,entertainment
2,gokidy,mom and children
3,cengizgumus_official,fashion
4,iamsiddeshjadhav,art
...,...,...
261,minikseyler_,mom and children
262,swothospitality,entertainment
263,brixtonturkiye,travel
264,utkucubukcuoglu,health and lifestyle


In [112]:
df_test_to_predict = df_test_m2[~df_test_m2['username'].isin(df_test_correct['username'])].reset_index(drop=True)
df_test_to_predict = df_test_to_predict[['username','category_enum','embeds_wavg']]
df_test_to_predict

,username,category_enum,embeds_wavg
0,livapastanesi,DESSERT_RESTAURANT,"[-0.021919882393740906, 0.011356396167832213, ..."
1,tusasshop,PRODUCT_SERVICE,"[-0.019723006614887892, 0.004805522574673402, ..."
2,etolyadigital,MARKETING_AGENCY,"[-0.01758344073235769, -0.011251978524460625, ..."
3,tulugozlu,ENTREPRENEUR,"[-5.637894897369102e-05, 0.007427698825628458,..."
4,krossbisiklet,BIKE_SHOP,"[0.008808300341866573, 0.023933282403852854, -..."
...,...,...,...
729,aynurtartan,PERSON,"[-0.004974343202574878, 0.011075228926762458, ..."
730,reklamcikafasi,NEWS_SITE,"[-0.006507353880531392, 0.00908726857508042, -..."
731,ucan_baba,DIGITAL_CREATOR,"[0.005686445175323538, -0.006737246289064363, ..."
732,rehberle,TOUR_GUIDE,"[-0.00048104971479939805, 0.002966425380183890..."


In [113]:
category_encoded_test = target_encoder.transform(df_test_to_predict["category_enum"].to_numpy().reshape(-1, 1))
df_test_to_predict["category_encoded"] = [row.tolist() for row in category_encoded_test]
df_test_to_predict



,username,category_enum,embeds_wavg,category_encoded
0,livapastanesi,DESSERT_RESTAURANT,"[-0.021919882393740906, 0.011356396167832213, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,tusasshop,PRODUCT_SERVICE,"[-0.019723006614887892, 0.004805522574673402, ...","[0.05646590752949935, 0.02575451918224273, 0.0..."
2,etolyadigital,MARKETING_AGENCY,"[-0.01758344073235769, -0.011251978524460625, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.563..."
3,tulugozlu,ENTREPRENEUR,"[-5.637894897369102e-05, 0.007427698825628458,...","[0.032830521237530226, 0.0973304984526431, 0.0..."
4,krossbisiklet,BIKE_SHOP,"[0.008808300341866573, 0.023933282403852854, -...","[0.06968259759211966, 0.11784020430499817, 0.1..."
...,...,...,...,...
729,aynurtartan,PERSON,"[-0.004974343202574878, 0.011075228926762458, ...","[0.04202162395774519, 0.18543621836174545, 0.1..."
730,reklamcikafasi,NEWS_SITE,"[-0.006507353880531392, 0.00908726857508042, -...","[0.09630410461467465, 0.26938586151560423, 0.0..."
731,ucan_baba,DIGITAL_CREATOR,"[0.005686445175323538, -0.006737246289064363, ...","[0.06524761388274747, 0.11595549521869573, 0.1..."
732,rehberle,TOUR_GUIDE,"[-0.00048104971479939805, 0.002966425380183890...","[0.06968259759211966, 0.11784020430499817, 0.1..."


In [114]:
X_test = pd.concat(
    [
        pd.DataFrame(df_test_to_predict["category_encoded"].tolist()),
        pd.DataFrame(df_test_to_predict["embeds_wavg"].tolist()),
    ],
    axis=1,
)

X_test

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.001356,0.006559,0.006988,-0.002775,0.004253,0.020998,0.002843,-0.003629,-0.005557,-0.009333
1,0.056466,0.025755,0.097490,0.322074,0.000000,0.077181,0.020581,0.015435,0.321095,0.061679,...,0.003983,0.021504,-0.000587,0.002128,0.009515,0.014089,-0.009502,0.008138,-0.008497,0.001748
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.563967,0.203089,...,0.007329,0.006978,0.010386,0.002290,0.000235,0.027875,-0.004651,0.003906,-0.018272,-0.002378
3,0.032831,0.097330,0.065391,0.316534,0.000000,0.161910,0.152923,0.063423,0.033108,0.065368,...,0.011217,0.017007,-0.002762,0.012900,-0.003544,0.011480,-0.009648,0.012897,-0.012853,0.006098
4,0.069683,0.117840,0.109084,0.186428,0.004743,0.183145,0.054360,0.041226,0.126231,0.107260,...,0.005898,-0.003711,-0.017707,0.009809,0.014664,0.023150,0.008364,0.005891,0.002824,0.002731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,0.042022,0.185436,0.104263,0.105208,0.019504,0.187410,0.122002,0.061863,0.083943,0.083724,...,-0.002453,0.006009,0.001576,0.004843,0.005238,0.022112,-0.002169,0.000105,-0.011948,0.000127
730,0.096304,0.269386,0.000000,0.000000,0.000000,0.198372,0.000000,0.154249,0.101979,0.100624,...,-0.000551,0.006297,0.002413,0.015238,0.007826,0.021905,-0.009672,-0.010606,-0.005435,-0.004599
731,0.065248,0.115955,0.151763,0.116286,0.007219,0.152366,0.057943,0.036264,0.072700,0.223111,...,0.000734,0.003797,0.010773,0.005863,0.002239,0.018910,-0.004231,0.007183,-0.010418,0.005263
732,0.069683,0.117840,0.109084,0.186428,0.004743,0.183145,0.054360,0.041226,0.126231,0.107260,...,0.004467,0.002055,0.001287,0.002757,0.005082,0.025245,-0.009223,0.002952,-0.016270,0.010015


In [115]:
X_test_numpy = X_test.to_numpy()

In [116]:
import xgboost as xgb

dtest = xgb.DMatrix(X_test_numpy)

model = xgb.Booster()
model.load_model('trained_models/xgb_openai_centroid.json')

y_pred = model.predict(dtest).astype(int)
y_pred

array([3, 8, 8, 9, 7, 3, 8, 1, 1, 7, 5, 8, 0, 5, 3, 5, 8, 0, 1, 2, 5, 3,
       5, 8, 8, 1, 5, 8, 0, 1, 1, 2, 6, 9, 2, 8, 5, 5, 5, 5, 3, 5, 1, 9,
       1, 3, 8, 2, 3, 2, 5, 1, 5, 6, 1, 3, 0, 9, 8, 5, 2, 5, 5, 5, 8, 7,
       3, 5, 0, 1, 6, 0, 8, 5, 9, 5, 9, 1, 5, 5, 9, 5, 0, 1, 5, 3, 3, 5,
       1, 5, 0, 5, 8, 8, 9, 9, 3, 5, 1, 6, 1, 2, 3, 3, 5, 5, 2, 1, 3, 3,
       8, 0, 1, 6, 8, 8, 3, 0, 8, 5, 1, 5, 5, 5, 1, 6, 5, 5, 2, 5, 0, 5,
       5, 2, 3, 8, 8, 5, 5, 1, 5, 5, 8, 8, 1, 5, 3, 2, 5, 9, 3, 3, 9, 0,
       3, 5, 1, 5, 5, 2, 0, 2, 9, 5, 5, 2, 1, 1, 3, 5, 3, 8, 1, 5, 3, 9,
       0, 1, 5, 5, 8, 9, 8, 8, 3, 2, 1, 1, 1, 5, 2, 8, 5, 3, 0, 5, 8, 8,
       0, 3, 6, 0, 8, 5, 5, 3, 9, 1, 1, 8, 1, 8, 5, 9, 1, 2, 2, 6, 7, 7,
       0, 9, 3, 8, 8, 1, 8, 5, 5, 1, 1, 5, 0, 0, 7, 5, 0, 5, 3, 5, 9, 9,
       2, 8, 1, 3, 5, 0, 3, 0, 5, 8, 1, 5, 3, 8, 0, 2, 0, 8, 8, 5, 2, 9,
       8, 3, 5, 3, 3, 1, 1, 8, 5, 7, 0, 1, 5, 3, 1, 6, 8, 5, 2, 3, 8, 2,
       8, 9, 9, 9, 2, 7, 8, 3, 5, 1, 2, 1, 1, 5, 2,

In [117]:
from sklearn.preprocessing import LabelEncoder

labels = [
    "entertainment",
    "food",
    "travel",
    "health and lifestyle",
    "mom and children",
    "fashion",
    "tech",
    "sports",
    "art",
    "gaming",
]

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_pred_labels = label_encoder.inverse_transform(y_pred)
y_pred_labels

array(['food', 'tech', 'tech', 'travel', 'sports', 'food', 'tech',
       'entertainment', 'entertainment', 'sports', 'health and lifestyle',
       'tech', 'art', 'health and lifestyle', 'food',
       'health and lifestyle', 'tech', 'art', 'entertainment', 'fashion',
       'health and lifestyle', 'food', 'health and lifestyle', 'tech',
       'tech', 'entertainment', 'health and lifestyle', 'tech', 'art',
       'entertainment', 'entertainment', 'fashion', 'mom and children',
       'travel', 'fashion', 'tech', 'health and lifestyle',
       'health and lifestyle', 'health and lifestyle',
       'health and lifestyle', 'food', 'health and lifestyle',
       'entertainment', 'travel', 'entertainment', 'food', 'tech',
       'fashion', 'food', 'fashion', 'health and lifestyle',
       'entertainment', 'health and lifestyle', 'mom and children',
       'entertainment', 'food', 'art', 'travel', 'tech',
       'health and lifestyle', 'fashion', 'health and lifestyle',
       'health and 

In [118]:
df_test_predicted = pd.concat([df_test_to_predict['username'], pd.DataFrame(y_pred_labels)], axis=1)
df_test_predicted.columns = ['username','label']
df_test_predicted

,username,label
0,livapastanesi,food
1,tusasshop,tech
2,etolyadigital,tech
3,tulugozlu,travel
4,krossbisiklet,sports
...,...,...
729,aynurtartan,health and lifestyle
730,reklamcikafasi,entertainment
731,ucan_baba,health and lifestyle
732,rehberle,travel


In [119]:
df_predicted = pd.concat([df_test_predicted, df_test_correct], axis=0).reset_index(drop=True)
df_predicted

,username,label
0,livapastanesi,food
1,tusasshop,tech
2,etolyadigital,tech
3,tulugozlu,travel
4,krossbisiklet,sports
...,...,...
995,minikseyler_,mom and children
996,swothospitality,entertainment
997,brixtonturkiye,travel
998,utkucubukcuoglu,health and lifestyle


In [120]:
label_conv = {
    "entertainment":'Entertainment',
    "food":'Food',
    "travel":'Travel',
    "health and lifestyle":'Health and Lifestyle',
    "mom and children":'Mom and Children',
    "fashion":'Fashion',
    "tech":'Tech',
    "sports":'Sports',
    "art":'Art',
    "gaming":'Gaming',
}

def correct_label(label):

    return label_conv[label]

df_predicted['label_correct'] = df_predicted['label'].apply(correct_label)
df_predicted = df_predicted.drop(columns=['label'])
df_predicted.columns = ['username','label']
df_predicted


,username,label
0,livapastanesi,Food
1,tusasshop,Tech
2,etolyadigital,Tech
3,tulugozlu,Travel
4,krossbisiklet,Sports
...,...,...
995,minikseyler_,Mom and Children
996,swothospitality,Entertainment
997,brixtonturkiye,Travel
998,utkucubukcuoglu,Health and Lifestyle


In [121]:
import json

result_dict = df_predicted.set_index('username')['label'].to_dict()
open('predictions/prediction-classification-round3.json','w').write(json.dumps(result_dict,indent=4))

35143